In [1]:
import pandas as pd
import datetime as dt
import re as re

In [2]:
#the event is depending on the barcode scanned afterwards

def activity_dependency(barcode):
    switcher = {
        'Production Start': 'Factory Closed',
        'New Recipe Start': 'Preparation/Changeover',
        'Break End': 'Break',
        'Production Finish': 'TBD'
    }
    return switcher.get(barcode, 'Production')

In [3]:
# 8 days of Delivery/Production Batch -- code 7 "Fri-NI Delivery" is not currently in use

def batchcode_dependency(batchcoded):
    switcher = {
        1: 'Sat-NI Delivery',
        2: 'Sun-NI Delivery',
        3: 'Mon-NI Delivery',
        4: 'Tue-NI Delivery',
        5: 'Wed-NI Delivery',
        6: 'Thu-NI Delivery',
        7: 'Fri-NI Delivery',
        8: 'Sun-SI Delivery',
        9: 'Tue-SI Delivery'
    }
    return switcher.get(batchcoded, '')

In [4]:
# morning shift -- 5:00 to 14:00
# afternoon shift -- 14:00 to 23:00
# Error -- Events compeleted in other time periods

def categorize_event_shift(timestamp):
    weekday_index = pd.Timestamp(timestamp).weekday()
    switcher = {
        0: 'Monday',
        1: 'Tuesday',
        2: 'Wednesday',
        3: 'Thursday',
        4: 'Friday',
        5: 'Saturday',
        6: 'Sunday'
    }
    weekday = switcher.get(weekday_index,'')
    hour = pd.Timestamp(timestamp).hour
    if hour in range(5,14):
        shift = 'Morning'
    elif hour in range(14,23):
        shift = 'Afternoon'
    else:
        shift = 'Error'
    return weekday +' '+ shift

In [5]:
#calculting time cost, sec as unit

def time_consumption(timestamp1,timestamp2):
    t1 = pd.to_datetime(timestamp1)
    t2 = pd.to_datetime(timestamp2)
    InMin = pd.Timedelta(t2-t1).seconds/60.0
    return format(InMin, '.2f')


In [6]:
# count how many pickers for ever events (+2 to get the total amount of people on this kitting line)

def count_pickers(picker):
    pattern = re.compile(r';|_|,')  ## <- temporary solutions overhere, may change according to the labels printed
    picker_array = pattern.findall(picker)  
    return len(picker_array)

In [7]:
staged_df = pd.read_csv(r'C:\Users\Peiran Quan\Desktop\python_data_preparation\staged.csv')  # Can change file path here
staged_df.head()

,Barcode,Production Batch,Recipe and P,Timestamp,Date,Seq Code,Week,Team Leader,Replenisher,Pickers,Break Reasons,Missing Products,Kitting Line,Timestamp and Date
0,Production Start,NaN,NaN,06:09:20,2021-03-25 00:00:00,NaN,NaN,iXUsr_Strzeletz(6101),iXUsr_Yang(7418),iXUsr_Julie(6746)iXUsr_Fane(6793)iXUsr_Kaur(67...,NaN,NaN,KL1,2021-03-25 06:09:20
1,New Recipe Start,NaN,NaN,06:09:32,2021-03-25 00:00:00,NaN,NaN,iXUsr_Strzeletz(6101),iXUsr_Yang(7418),iXUsr_Julie(6746)iXUsr_Fane(6793)iXUsr_Kaur(67...,NaN,NaN,KL1,2021-03-25 06:09:32
2,813U4$$$$$$$$0001000301,8.0,U4,06:13:21,2021-03-25 00:00:00,0001 of 0003,13.0,iXUsr_Strzeletz(6101),iXUsr_Yang(7418),iXUsr_Julie(6746)iXUsr_Fane(6793)iXUsr_Kaur(67...,NaN,NaN,KL1,2021-03-25 06:13:21
3,813U4$$$$$$$$0002000301,8.0,U4,06:17:47,2021-03-25 00:00:00,0002 of 0003,13.0,iXUsr_Strzeletz(6101),iXUsr_Yang(7418),iXUsr_Julie(6746)iXUsr_Fane(6793)iXUsr_Kaur(67...,NaN,NaN,KL1,2021-03-25 06:17:47
4,813U4$$$$$$$$0003000301,8.0,U4,06:20:48,2021-03-25 00:00:00,0003 of 0003,13.0,iXUsr_Strzeletz(6101),iXUsr_Yang(7418),iXUsr_Julie(6746)iXUsr_Fane(6793)iXUsr_Kaur(67...,NaN,NaN,KL1,2021-03-25 06:20:48


In [8]:
# copy raw df
df_final = pd.DataFrame(data=None) #df is the event data frame

In [9]:
def main(df_temp_raw):
    df_temp_event = pd.DataFrame(columns=('Start Time', 'Finish Time', 'Activity', 'Seq Code' ,'Recipe Name', 'Break Reasons', 'Missing Ingredients', 'Kitting Line', 'Assembly Batch', 'Event Shift', 'Team Leader', 'Pickers Count', 'Time Consumption'))
    index_align = df_temp_raw.first_valid_index() # pandas is using the df_temp_raw's frame index whenever df_temp_raw is called
    df_temp_event['Finish Time'] = df_temp_raw['Timestamp and Date']
    df_temp_event['Seq Code'] = df_temp_raw['Seq Code']
    df_temp_event['Start Time'].loc[index_align] = '2019-1-1 00:00:00'
    #df_temp_event['Recipe Name'][1] = df_temp_raw['Recipe and P'].loc[df_temp_raw['Recipe and P'].first_valid_index()]
    #print(df_temp_event)
    df_temp_raw.loc[index_align, 'Recipe and P'] = df_temp_raw['Recipe and P'].loc[df_temp_raw['Recipe and P'].first_valid_index()] #ignore the warning -- data type is not changing, the return is in VIEW

    for i in range (len(df_temp_raw)):
        df_temp_event['Start Time'].loc[index_align+i+1] = df_temp_raw['Timestamp and Date'].loc[index_align+i]
        df_temp_event['Activity'].loc[index_align+i] = activity_dependency(df_temp_raw['Barcode'].loc[index_align+i])
        df_temp_event['Break Reasons'].loc[index_align+i+1] = df_temp_raw['Break Reasons'].loc[index_align+i]
        df_temp_event['Missing Ingredients'].loc[index_align+i+1] = df_temp_raw['Missing Products'].loc[index_align+i]
        df_temp_event['Kitting Line'].loc[index_align+i+1] = df_temp_raw['Kitting Line'].loc[index_align+i]
        df_temp_event['Assembly Batch'].loc[index_align+i] = batchcode_dependency(df_temp_raw['Production Batch'].loc[index_align+i])
        df_temp_event['Time Consumption'].loc[index_align+i] = time_consumption(df_temp_event['Start Time'].loc[index_align+i], df_temp_event['Finish Time'].loc[index_align+i])
        df_temp_event['Event Shift'].loc[index_align+i] = categorize_event_shift(df_temp_event['Finish Time'].loc[index_align+i])
        df_temp_event['Team Leader'].loc[index_align+i] = df_temp_raw['Team Leader'].loc[index_align+i]
        df_temp_event['Pickers Count'].loc[index_align+i] = count_pickers(df_temp_raw['Pickers'].loc[index_align+i])
        #print(df_temp_event)
        """
        for j in range (len(df_temp_raw)):  ############################################# slow, deprecated
            print(df_temp_raw['Recipe and P'][::-1].loc[(index_align + j):])
            reversed_index = df_temp_raw['Recipe and P'][::-1].loc[(index_align + j):].first_valid_index()
        """
        #df_temp_raw['Recipe and P'][index_align] = df_temp_raw['Recipe and P'].loc[df_temp_raw['Recipe and P'].first_valid_index()]
        recipe_index = df_temp_raw['Recipe and P'].loc[0:index_align+i].last_valid_index()
        #print(recipe_index)
        df_temp_event['Recipe Name'].loc[index_align+i] = df_temp_raw['Recipe and P'].loc[recipe_index]
        #print(df_temp_raw['Recipe and P'].loc[recipe_index])
    #print(df_temp_event)
    return df_temp_event

In [10]:
for kl in range(1,21):
    df_temp_raw = staged_df.loc[staged_df['Kitting Line'] == 'KL%s'%kl]
    if df_temp_raw.empty:
        print("Kitting Line %r is not included!" %(kl))  # if data frame is empty
    #print(df_temp_raw)
    else:
        seg = main(df_temp_raw)
        #print(seg)
        df_final = pd.concat([df_final, seg], sort = False)
        print("Kitting Line %r Completed!" %(kl))
# here we have several different ways to validate the logic

Kitting Line 1 Completed!
Kitting Line 2 Completed!
Kitting Line 3 Completed!
Kitting Line 4 Completed!
Kitting Line 5 Completed!
Kitting Line 6 Completed!
Kitting Line 7 Completed!
Kitting Line 8 Completed!
Kitting Line 9 Completed!
Kitting Line 10 Completed!
Kitting Line 11 Completed!
Kitting Line 12 Completed!
Kitting Line 13 Completed!
Kitting Line 14 Completed!
Kitting Line 15 Completed!
Kitting Line 16 Completed!
Kitting Line 17 Completed!
Kitting Line 18 Completed!
Kitting Line 19 Completed!
Kitting Line 20 Completed!


In [11]:
## Attach a column to identify the Week
## Update weekly in the format of YYYY-WW 
df_final['Week'] = '2021-13'
df_final.tail()

,Start Time,Finish Time,Activity,Seq Code,Recipe Name,Break Reasons,Missing Ingredients,Kitting Line,Assembly Batch,Event Shift,Team Leader,Pickers Count,Time Consumption,Week
7064,2021-03-30 07:18:12,2021-03-30 07:20:21,Production,0005 of 0005,Q2,NaN,NaN,KL20,Wed-NI Delivery,Tuesday Morning,iXUsr_Chang(8908),5,2.15,2021-13
7065,2021-03-30 07:20:21,2021-03-30 07:40:22,Preparation/Changeover,NaN,Q2,NaN,NaN,KL20,,Tuesday Morning,iXUsr_Chang(8908),5,20.02,2021-13
7066,2021-03-30 07:40:22,2021-03-30 07:43:55,Production,0001 of 0002,X2,NaN,NaN,KL20,Wed-NI Delivery,Tuesday Morning,iXUsr_Chang(8908),5,3.55,2021-13
7067,2021-03-30 07:43:55,2021-03-30 07:44:41,Production,0002 of 0002,X2,NaN,NaN,KL20,Wed-NI Delivery,Tuesday Morning,iXUsr_Chang(8908),5,0.77,2021-13
7068,2021-03-30 07:44:41,2021-03-30 07:45:03,TBD,NaN,X2,NaN,NaN,KL20,,Tuesday Morning,iXUsr_Chang(8908),5,0.37,2021-13


In [12]:
df_final.to_csv(r'C:\Users\Peiran Quan\Desktop\python_data_preparation\prepared.csv', index = False) # Can change file path here

In [13]:
df_final['Start Time'].astype('datetime64')

0      2019-01-01 00:00:00
1      2021-03-25 06:09:20
2      2021-03-25 06:09:32
3      2021-03-25 06:13:21
4      2021-03-25 06:17:47
               ...        
7064   2021-03-30 07:18:12
7065   2021-03-30 07:20:21
7066   2021-03-30 07:40:22
7067   2021-03-30 07:43:55
7068   2021-03-30 07:44:41
Name: Start Time, Length: 7069, dtype: datetime64[ns]